In [7]:
# General libraries
import pandas as pd
import geopandas as gpd
import regex as re


In [2]:
central_cases = gpd.read_file("data/dengue-cases-central-geojson.geojson")

In [21]:
sum(central_cases.Description.map(lambda x: int(re.search(r"(<td>.*?</td>)", x).group(0)[4:-5])))

807

In [32]:
import tweepy as tw
import pprint

from keys import (
    twitter_consumer_key,
    twitter_consumer_secret,
    twitter_access_token,
    twitter_access_token_secret,
)

auth = tw.OAuthHandler(twitter_consumer_key, twitter_consumer_secret)
auth.set_access_token(twitter_access_token, twitter_access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [38]:
search_term = "dengue -filter:retweets"

tweets = tw.Cursor(api.search,
                   q=search_term,
                   lang="en",
                   since='2018-04-23').items(1)

all_tweets = [tweet for tweet in tweets]
pprint.pprint(all_tweets[0]._json)

{'contributors': None,
 'coordinates': None,
 'created_at': 'Thu Aug 06 15:18:49 +0000 2020',
 'entities': {'hashtags': [],
              'symbols': [],
              'urls': [{'display_url': 'news18.com/news/india/31-…',
                        'expanded_url': 'https://www.news18.com/news/india/31-dengue-cases-in-delhi-till-august-1-fever-clinics-set-up-campaigns-intensified-2764327.html',
                        'indices': [93, 116],
                        'url': 'https://t.co/xyNbt9TYtG'}],
              'user_mentions': []},
 'favorite_count': 0,
 'favorited': False,
 'geo': None,
 'id': 1291393298683162626,
 'id_str': '1291393298683162626',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'en',
 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
 'place': None,
 'possibly_sensitive': False,
 'retweet_count': 0,
 'r